In [182]:
# the viz stuff
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display

# cause data
import pandas as pd
import numpy as np

# talk to the web
import requests

# api keys are in config_air.ini to keep them outside of this public notebook
import configparser

# First, reading in the api keys

doing it so the code can be pushed to github without plastering api keys all over the interwebs

In [183]:
config = configparser.ConfigParser()
config.read('config_air.ini')

try:
    air_visual_api_key = config['AirVisual']["api_key"]
except KeyError:
    print("Missing Key AirVisual in the config_air.ini file")

if air_visual_api_key is not None:
    print("Successfully imported airvisual api key")

Successfully imported airvisual api key


# getting the air quality data from air visual

AirVisual stations in Pakistan: https://airvisual.com/pakistan

In [200]:
# http://api.airvisual.com/v2/nearest_city?lat=Value&lon=Value&rad=Value&key=YourAPIKey
    
lat, long = 24.861462, 67.009939 # coordinates for Karachi
url = "http://api.airvisual.com/v2/nearest_city"
payload = {"lat": lat, "lon": long, "rad": 20, "key": air_visual_api_key}
r = requests.get(url, params=payload)
print(r.text)

{"status":"success","data":{"city":"Karachi","state":"Sindh","country":"Pakistan","location":{"type":"Point","coordinates":[67.05294147133827,24.818095393631435]},"current":{"weather":{"ts":"2017-09-26T03:00:00.000Z","tp":27,"pr":1008,"hu":74,"ws":1,"ic":"03d"},"pollution":{"ts":"2017-09-26T04:00:00.000Z","aqius":54,"mainus":"p2","aqicn":19,"maincn":"p2"}}}}


Lets make a function out of this which returns the data dic:

In [221]:
cities_loc["lahore"]

('31.5195822', '74.326297')

In [287]:
def get_air_quality(lat=24.861462, long=67.009939, rad=30):
    """takes in lat, long and returns air quality 
       defaults to Karachi if no lat/long passed in"""
    url = "http://api.airvisual.com/v2/nearest_city"
    payload = {"lat": lat, "lon": long, "rad": rad, "key": air_visual_api_key}
    r = requests.get(url, params=payload)
    
    if r.status_code == requests.codes.ok:
        return r.json()["data"]
    else:
        return "something went wrong"

q = get_air_quality()
print(q["city"])
q["current"]["pollution"]

Karachi


{'aqicn': 20,
 'aqius': 54,
 'maincn': 'p2',
 'mainus': 'p2',
 'ts': '2017-09-26T05:00:00.000Z'}

# getting lat/long from a city name

Ideally we want the user to type in a city and it looks for that:

In [288]:
def get_lat_long(query="Karachi"):
    """takes in a city name and returns lat long
       usin osm lookup"""
    
    # return from dict if already looked this up
    if query.lower() in cities_loc:
        return cities_loc[query.lower()]
    
    # find the lat long 
    url = "http://nominatim.openstreetmap.org/search"
    payload = {"format": "json", "q": query, "country": "Pakistan", "limit": 1}
    r = requests.get(url, params=payload)
    
    try:
        d = r.json()[0]
    except:
        return False
    
    return float(d["lat"]), float(d["lon"])
    
cities = ["karachi", "lahore", "islamabad", "peshawar"]
cities_loc = {city: get_lat_long(city) for city in cities}
cities_loc

{'islamabad': (22.3307998, 91.8412863),
 'karachi': (25.147872, 67.1821931713384),
 'lahore': (31.5195822, 74.326297),
 'peshawar': (34.0123846, 71.5787458)}

# a comparision

In [289]:
def air_quality(query="Karachi"):
    """takes in a city name and returns the air data"""
    latitude, longitude = get_lat_long(query)
    q = get_air_quality(latitude, longitude)
    return q["city"], q["current"]["pollution"]
    
city, pollution = air_quality("peshawar")
city, pollution

('Peshawar',
 {'aqicn': 182,
  'aqius': 193,
  'maincn': 'p2',
  'mainus': 'p2',
  'ts': '2017-09-26T05:00:00.000Z'})

In [290]:
for city in cities:
    print(city)
    city, pollution = air_quality(city)
    print(pollution["aqius"])

karachi


TypeError: string indices must be integers